In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
.appName("ShobhitApp") \
.getOrCreate()

In [2]:
from pyspark.sql.functions import col,explode

In [3]:
%%sh
hdfs dfs -cat /user/itv000197/test2.json

{
  "status": true,
  "data": {
    "title": "Top By Daily Interest",
    "phones": [
      {
        "phone_name": "Samsung Galaxy A53 5G",
        "slug": "samsung_galaxy_a53_5g-11268",
        "hits": 30283,
        "detail": "http://api-mobilespecs.azharimm.site/v2/samsung_galaxy_a53_5g-11268"
      },
      {
        "phone_name": "Samsung Galaxy S22 Ultra 5G",
        "slug": "samsung_galaxy_s22_ultra_5g-11251",
        "hits": 23673,
        "detail": "http://api-mobilespecs.azharimm.site/v2/samsung_galaxy_s22_ultra_5g-11251"
      },
      {
        "phone_name": "Xiaomi Redmi Note 11",
        "slug": "xiaomi_redmi_note_11-11336",
        "hits": 22833,
        "detail": "http://api-mobilespecs.azharimm.site/v2/xiaomi_redmi_note_11-11336"
      },
      {
        "phone_name": "Samsung Galaxy A73 5G",
        "slug": "samsung_galaxy_a73_5g-11257",
        "hits": 20059,
        "detail": "http://api-mobilespecs.azharimm.site/v2/samsung_galaxy_a73_5g-11257"
      },
      {
   

In [4]:
df = spark.read.option("multiline","true").json("/user/itv000197/test2.json")

In [5]:
#df.schema.json()
df.schema.simpleString()

'struct<data:struct<phones:array<struct<detail:string,hits:bigint,phone_name:string,slug:string>>,title:string>,status:boolean>'

In [6]:
df.printSchema()

root
 |-- data: struct (nullable = true)
 |    |-- phones: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- detail: string (nullable = true)
 |    |    |    |-- hits: long (nullable = true)
 |    |    |    |-- phone_name: string (nullable = true)
 |    |    |    |-- slug: string (nullable = true)
 |    |-- title: string (nullable = true)
 |-- status: boolean (nullable = true)



In [7]:
df_new = df.select("status","data.title","data.phones")\
.select( "status",col("title"),explode(col("phones")).alias("data_col"))\
.select("status","title","data_col.detail","data_col.hits","data_col.phone_name","data_col.slug")

In [8]:
df_new.show()

+------+--------------------+--------------------+-----+--------------------+--------------------+
|status|               title|              detail| hits|          phone_name|                slug|
+------+--------------------+--------------------+-----+--------------------+--------------------+
|  true|Top By Daily Inte...|http://api-mobile...|30283|Samsung Galaxy A5...|samsung_galaxy_a5...|
|  true|Top By Daily Inte...|http://api-mobile...|23673|Samsung Galaxy S2...|samsung_galaxy_s2...|
|  true|Top By Daily Inte...|http://api-mobile...|22833|Xiaomi Redmi Note 11|xiaomi_redmi_note...|
|  true|Top By Daily Inte...|http://api-mobile...|20059|Samsung Galaxy A7...|samsung_galaxy_a7...|
|  true|Top By Daily Inte...|http://api-mobile...|19082|   Xiaomi Poco F4 GT|xiaomi_poco_f4_gt...|
|  true|Top By Daily Inte...|http://api-mobile...|17431|         vivo X Note|   vivo_x_note-11469|
|  true|Top By Daily Inte...|http://api-mobile...|17203|       Xiaomi 12 Pro| xiaomi_12_pro-11287|
|  true|To